In [1]:
#!pip install --upgrade git+https://github.com/EmGarr/kerod.git

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import  functools
import  tensorflow as tf
import  tensorflow_datasets as tfds
from    tensorflow.keras.utils import to_categorical
import  matplotlib.pyplot as plt
from    tensorflow.keras import layers
from    tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import pandas as pd
from    tqdm import tqdm
import  time
from    sklearn.metrics import accuracy_score
import argparse

from common.config import Config
from common.logger import Logger
from common.communication import Client
from common.communication import Server
from common.helper import ImagesInfo 
from common.timekeeper import TimeKeeper
from common.helper import read_image, filt_text, get_predictions
from CaptionModel import CaptionModel
from common.helper import read_image, filt_text, get_predictions,process_predictions,get_reshape_size

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('-s', '--server', action='store', type=str, required=False)
parser.add_argument('-t', '--test_number', action='store', type=int, required=False)
parser.add_argument('-l', '--split_layer', action='store', type=int, required=False)
parser.add_argument('-v', '--verbose', action='store', type=int, required=False)
parser.add_argument('-i', '--image_size', action='store', type=int, required=False)
parser.add_argument('-m', '--max_tests', action='store', type=int, required=False)
args, unknown = parser.parse_known_args()

split_layer = args.split_layer

if(split_layer == None):
    split_layer = 3

Logger.milestone_print("Splitting at layer %d" % split_layer)

Splitting at layer 3


In [5]:
tf.get_logger().setLevel('ERROR')

In [6]:
N_LABELS = 80

data_dir='/home/suphale/coco'
split_train = "train[:1%]"
split_val = "validation[:1%]"
image_size = 250
h_image_height = image_size
h_image_width = image_size

In [7]:
tk = TimeKeeper()
cfg = Config()
client = Client(cfg)
imagesInfo = ImagesInfo(cfg)

In [8]:
from tensorflow import keras  # or import keras for standalone version
from tensorflow.keras.layers import Input

org_model = tf.keras.models.load_model(cfg.saved_model_path + '/iv3_full_model', compile=False)

# basic_model = tf.keras.models.load_model(cfg.saved_model_path + '/model')
# org_model = tf.keras.Model(inputs=basic_model.input, 
#         outputs=[basic_model.get_layer('mixed10').output, basic_model.get_layer('dense_1').output] )

In [9]:
model_config = org_model.get_config()

In [10]:
max_layer_index = len(model_config['layers']) - 1

In [11]:

head_model_config = {}
head_model_config['name'] = 'head_model'
head_model_config['layers'] = []
head_model_config['input_layers'] = [[model_config['layers'][0]['name'],0,0]]
head_model_config['output_layers'] = [[model_config['layers'][split_layer+1-1]['name'],0,0]]

for index in range(split_layer+1):
    head_model_config['layers'].append(model_config['layers'][index])

print("Final layer of head model [%d] %s" % (split_layer, model_config['layers'][split_layer]['name']) )


Final layer of head model [3] activation


In [12]:
# Last layer of the head model
last_head_model_layer = head_model_config['layers'][split_layer]['name']
# print(last_head_model_layer)

In [13]:
# First layer of the tail model
print("First layer of tail model [%d] %s" % (split_layer+1, model_config['layers'][split_layer+1]['name']) )


First layer of tail model [4] conv2d_1


In [14]:
import copy

tail_model_config = copy.deepcopy(model_config)
tail_model_config['name'] = 'tail_model'
tail_model_config['input_layers'] = [[model_config['layers'][split_layer+1]['name'],0,0]]
# tail_model_config['output_layers'] = [[model_config['layers'][max_layer_index]['name'],0,0]]
tail_model_config['output_layers'] = model_config['output_layers']

In [15]:
# print(tail_model_config['input_layers'])

In [16]:
# print(tail_model_config['output_layers'])

In [17]:
new_input_layer = {
                      'name': 'new_input',
                      'class_name': 'InputLayer',
                      'config': {
                          'batch_input_shape': tuple(org_model.layers[split_layer+1-1].output.shape),
                          'dtype': 'float32',
                          'sparse': False,
                          'name': 'new_input'
                      },
                      'inbound_nodes': []
                  }
tail_model_config['layers'][0] = new_input_layer

In [18]:
for index in range(1,split_layer+1):
    # print("%d %s" % (index, tail_model_config['layers'][1]['name']) )
    tail_model_config['layers'].pop(1)

In [19]:
import numpy as np

# Find if any layer in the tail model takes the last layer of head model as input
# substitute it with the input layer
# Ideally we should check if any tail model layer refers to any head model layer ToDo
for index, layer in enumerate(tail_model_config['layers']):
    if (np.shape(layer['inbound_nodes'])[0] > 0):
        dim_1 = len(layer['inbound_nodes'][0])
        if(dim_1 >= 1):
            for i in range(dim_1):
                in_layer = layer['inbound_nodes'][0][i][0]
                if(in_layer == last_head_model_layer):
                    print(str(index) + "    " + layer['name'] + " -> " + in_layer )
                    # print(tail_model_config['layers'][index]['inbound_nodes'][0][i])
                    tail_model_config['layers'][index]['inbound_nodes'][0][i] = [[['new_input', 0, 0, {}]]]


1    conv2d_1 -> activation


In [20]:
# tail_model_config['layers'][1]['inbound_nodes'] = [[['new_input', 0, 0, {}]]]
tail_model_config['input_layers'] = [['new_input', 0, 0]]

In [21]:
import pprint
with open(cfg.temp_path + '/model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(model_config))
    print(model_config,file=fh)
with open(cfg.temp_path + '/head_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    print(head_model_config,file=fh)
with open(cfg.temp_path + '/tail_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    print(tail_model_config,file=fh)


In [22]:
head_model = org_model.__class__.from_config(head_model_config, custom_objects={})

In [23]:
tail_model = org_model.__class__.from_config(tail_model_config, custom_objects={})

In [24]:
with open(cfg.temp_path + '/org_model.txt','w') as fh:
    org_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open(cfg.temp_path + '/head_model.txt','w') as fh:
    head_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open(cfg.temp_path + '/tail_model.txt','w') as fh:
    tail_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)


In [25]:
for index, layer in enumerate(org_model.layers[:split_layer+1]):
    # print("[%d] %s %s" % (index, layer.name, str(np.shape(weight))))
    weight = layer.get_weights()
    new_head_model_layer = head_model.layers[index]
    new_head_model_layer.set_weights(weight)

In [26]:
import numpy as np
for index, layer in enumerate(org_model.layers[split_layer+1:max_layer_index+1]):
    weight = layer.get_weights()
    # print("org_model [%d] %s %s" % (index, layer.name, str(tf.shape(weight))))
    tail_model_layer = tail_model.layers[index+1]
    tail_model_layer_weight = tail_model_layer.get_weights()
    # print("tail_model [%d] %s %s" % (index, tail_model_layer.name, str(tf.shape(tail_model_layer_weight))))
    tail_model_layer.set_weights(weight)

In [27]:
head_model.save(cfg.temp_path + '/iv3_head_model_'+str(split_layer))
tail_model.save(cfg.temp_path + '/iv3_tail_model_'+str(split_layer))

In [28]:
class BoxField:
    BOXES = 'bbox'
    KEYPOINTS = 'keypoints'
    LABELS = 'label'
    MASKS = 'masks'
    NUM_BOXES = 'num_boxes'
    SCORES = 'scores'
    WEIGHTS = 'weights'

class DatasetField:
    IMAGES = 'images'
    IMAGES_INFO = 'images_information'
    IMAGES_PMASK = 'images_padding_mask'

def my_preprocess(inputs):
    image = inputs['image']
    image = tf.image.resize(image, (h_image_height, h_image_width))
    image = tf.cast(image, tf.float32)
    image /= 127.5
    image -= 1.

    targets = inputs['objects']

    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32)

    inputs = {DatasetField.IMAGES: image, DatasetField.IMAGES_INFO: image_information}

    # ground_truths = {
    #     BoxField.BOXES: targets[BoxField.BOXES] * tf.tile(image_information[tf.newaxis], [1, 2]),
    #     BoxField.LABELS: tf.cast(targets[BoxField.LABELS], tf.int32),
    #     BoxField.NUM_BOXES: tf.shape(targets[BoxField.LABELS]),
    #     BoxField.WEIGHTS: tf.fill(tf.shape(targets[BoxField.LABELS]), 1.0)
    # }
    ground_truths = tf.cast(targets[BoxField.LABELS], tf.int32)
    ground_truths = tf.one_hot(ground_truths, depth=N_LABELS, dtype=tf.int32)
    ground_truths = tf.reduce_sum(ground_truths, 0)
    ground_truths = tf.greater( ground_truths, tf.constant( 0 ) )    
    ground_truths = tf.where (ground_truths, 1, 0) 
    return image, ground_truths

def expand_dims_for_single_batch(image, ground_truths):
    image = tf.expand_dims(image, axis=0)
    ground_truths = tf.expand_dims(ground_truths, axis=0)
    return image, ground_truths


In [29]:
ds_train, ds_info = tfds.load(name="coco/2017", split=split_train, data_dir=data_dir, shuffle_files=True, download=False, with_info=True)
ds_train = ds_train.map(functools.partial(my_preprocess), num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_val, ds_info = tfds.load(name="coco/2017", split=split_val, data_dir=data_dir, shuffle_files=False, download=False, with_info=True)
ds_val = ds_val.map(functools.partial(my_preprocess), num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [30]:
# org_model.save(cfg.temp_path + '/full_model')

In [34]:
classes = ds_info.features['objects']['label'].names
num_classes = len(classes)
print(num_classes)

80


In [84]:
from tqdm import tqdm

train_writer = tf.io.TFRecordWriter('train.tfrecord')
val_writer = tf.io.TFRecordWriter('val.tfrecord')

sample_feature = None

for test_index in tqdm(range(2)):
    sample_img_batch, sample_cap_batch = next(iter(ds_train))
    h = head_model(sample_img_batch)
    features, result = tail_model(h)

    n = h.numpy()
    print(tf.shape(n))

    # example = tf.train.Example(features = tf.train.Features(feature = {
    #         'filename': tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes(filename, 'utf-8') ])),
    #         'Image shapes': tf.train.Feature(int64_list = tf.train.Int64List(value = [img_shape[0],img_shape[1]])),
    #         'image': tf.train.Feature(bytes_list = tf.train.BytesList(value = [image])),
    #         'label': tf.train.Feature(bytes_list = tf.train.BytesList(value = [label]))}))

    sample_feature={
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[n.astype(np.int64).tobytes()])),
        # 'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[sample_cap_batch])),
    }

    example = tf.train.Example(features=tf.train.Features(feature=sample_feature))
    train_writer.write(example.SerializeToString())

    if(False):
        print("Reference  : ", end=' ')
        n = sample_cap_batch.numpy()
        index = 0
        for x in n[0]:
            if x > 0.1:
                print("%s," % (classes[index]), end=' ')
            index += 1
        print("")
        print("Prediction : ", end=' ')
        n = result.numpy()
        index = 0
        for x in n[0]:
            if x > 0.5:
                print("%s(%.2f)," % (classes[index],x), end=' ')
            index += 1
        print("")

 50%|█████     | 1/2 [00:00<00:00,  1.17it/s]

tf.Tensor([  1 124 124  32], shape=(4,), dtype=int32)


100%|██████████| 2/2 [00:01<00:00,  1.05it/s]

tf.Tensor([  1 124 124  32], shape=(4,), dtype=int32)


In [85]:
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.int64),
    # 'image': tf.train.Feature(bytes_list = tf.train.BytesList(value = [image]))
    # 'label': tf.io.FixedLenFeature([], tf.int32),
}

def _parse_image_function(example_proto):
    features = tf.io.parse_single_example(example_proto, sample_feature)
    image = tf.io.decode_raw(features['image'], tf.int64)
    # label = tf.io.decode_raw(features['image'], tf.uint8)

    return image, image

In [86]:
def read_dataset(epochs, batch_size, channel, channel_name):

    filenames = [os.path.join(channel, channel_name + '.tfrecord')]
    dataset = tf.data.TFRecordDataset(filenames)


    dataset = dataset.map(_parse_image_function, num_parallel_calls=10)
    dataset = dataset.prefetch(10)
    dataset = dataset.repeat(epochs)
    dataset = dataset.shuffle(buffer_size=10 * batch_size)
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset

In [87]:
train_dataset = read_dataset(1, 10, '.', 'train')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [67]:
image_batch, label_batch = next(iter(train_dataset))

DataLossError: truncated record at 194848929' failed with Read less bytes than requested